In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from viewing import *
from util import *

# Configuration of data frames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

# Configuration of matplotlib and seaborn
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize']= 16
plt.rcParams['axes.titlesize']= 18
plt.rcParams['axes.labelsize']= 18 
plt.rcParams['figure.titlesize']= 18
plt.rcParams['figure.figsize'] = [28, 14]  # Scale up plots

# CONTROL E IMPLEMENTACIÓN DE UNA PUF EN UN MICROPROCESADOR
## Sergio Vinagrero Gutiérrez

# Parámetros de identificación de una placa

El ID de un microprocesador viene implantado de fabrica y se compone de 96 bits.
En dichos bits están codificados, por orden:

+ Oblea: 1 bytes
+ Lote: 7 bytes
+ Coordenada X en la oblea: 2 bytes BCD
+ Coordenada Y en la oblea: 2 bytes BCD

In [2]:
boards_ids_df = pd.read_csv('./csv/boards_ids.csv', index_col=None)
boards_ids = boards_ids_df['Board'].values.tolist()

boards_params_df = pd.read_csv('./csv/boards_params.csv', index_col=None)

In [3]:
boards_params_df.head(10)

,Board,Wafer,Lot,X,Y
0,0x3031471737343534440230,30,31471737343534,35,68
1,0x343047336733632A303C0,34,3047336733632A,192,48
2,0x3430471836733632330B70,34,30471836733632,183,51
3,0x3430476367336321F0360,34,30476367336321,96,240
4,0x3430473367336322F13C0,34,30473367336322,192,241
5,0x3530471436733632E0440,35,30471436733632,64,224
6,0x3031471737343534300460,30,31471737343534,70,48
7,0x3430471836733632270410,34,30471836733632,65,39
8,0x3430471836733632440430,34,30471836733632,67,68
9,0x30314718373435342A02B0,30,31471837343534,43,42


# Muestras de regiones de memoria

Se han extraido muestras de 512 bytes de cada región de memoria.
Por cada placa se han tomado por lo tanto 32 muestras.

Como necesitamos al menos 2 muestras de la misma zona de memoria, hay un mínimo de 2 tandas de muestras por cada placa.

## Estructura de una muestra

En cada muestra de se toman los siguientes valores:

+ ID del microprocesador
+ Posición de la memoria que se va a leer.
+ Temperatura interna del microprocesador.
+ Tensión de referencia del microprocesador.
+ Datos de calibración del microprocesador a 30 y 110 grados.
+ Datos de calibración del microprocesador a 3.3 voltios.
+ Fecha y hora de la realización de la muestra
+ 512 bytes

In [4]:
diffs_regions_all_df = pd.read_csv('./csv/diffs_regions_all_boards.csv', index_col=None)
diffs_regions_all_df.sort_values(['Region'], ascending=[True], inplace=True)

diffs_regions_summary_df = pd.read_csv('./csv/diffs_regions_all_boards_summary.csv')

# Diferencias entre muestras de la misma placa

Para cada placa, se han comparado los 512 bytes de las mismas zonas entre sí.

In [5]:
diffs_regions_summary_df

,Region,Mean_Diff,Median,Std_Dev,Num_Samples
0,0x20000000,1.122126,0.976562,0.621462,212
1,0x20000200,12.022774,0.000000,21.719085,212
2,0x20000400,23.792195,0.000000,41.838124,212
3,0x20000600,38.552108,14.843750,35.644271,212
4,0x20000800,60.896963,37.304688,36.672386,212
5,0x20000a00,60.981722,37.109375,36.598932,212
6,0x20000c00,59.972914,35.546875,37.170696,212
7,0x20000e00,60.706257,38.183594,36.549096,212
8,0x20001000,61.271005,37.109375,36.414213,212
9,0x20001200,60.947634,37.109375,36.794650,212


In [ ]:
# Diferencias de muestras entre placas